In [20]:
# read in pdf and extract text
# We want to ignore all the decorum and only extract the text (i.e. no page number etc.)
import pytesseract
from PyPDF2 import PdfReader
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
import fitz
from PIL import Image

from io import StringIO
import re
import os

In [21]:
pytesseract.pytesseract.tesseract_cmd = 'C:/Program Files/Tesseract-OCR/tesseract.exe'

In [22]:
pdf_path = '//Ds/data/LAB/laupodteam/AIOS/Bram/language_modeling/MEDICAL_TEXT/RAW/PhDTheses'

In [45]:
def pdf_to_text(path, backends=['pypdf', 'fitz']):
    '''Extract text from pdf documents
        Source: https://towardsdatascience.com/pdf-preprocessing-with-python-19829752af9f
    '''
    def pdfminer(_path):
        manager = PDFResourceManager()
        retstr = StringIO()
        layout = LAParams(all_texts=False, detect_vertical=True)
        device = TextConverter(manager, retstr, laparams=layout)
        interpreter = PDFPageInterpreter(manager, device)
        with open(_path, 'rb') as filepath:
            for page in PDFPage.get_pages(filepath, check_extractable=True):
                interpreter.process_page(page)
        text = retstr.getvalue()
        device.close()
        retstr.close()
        return text

    def pypdfer(_path):
        reader = PdfReader(_path)
        return [p.extract_text(0) for p in reader.pages]
    
    error = ""

    if 'pypdf' in backends:
        try:
            return pypdfer(path), None
        except Exception as e_1:
            error = error + f"\n PyPDF failed: {e_1}"
   
    if 'fitz' in backends:
        try:
            pdf_file = fitz.open(path)
            pages = []
            for _p in pdf_file:
                pages.append(_p.getText())
            return pages, error
        except Exception as e_2:
            error = error + f"\n PyMuPDF failed: {e_2}"
        
    if 'pdfminer' in backends:
        try:
            return pdfminer(path), None
        except Exception as e_3:
            error = error + f"pdfminer failed: {e_3}"

    if 'pytesseract' in backends:
        try:
            return pytesseract.image_to_string(path, lang='en'), None
        except Exception as e_4:
            return error + f"\n PyTesseract failed: {e_4}" 
        
# https://arxiv.org/abs/2308.13418

In [ ]:
Files = [f for f in os.listdir(pdf_path) if f.endswith('.pdf')]

In [46]:
_File = 'Radboud_47910.pdf' # Files[245]
_path = os.path.join(pdf_path, _File)
Text, error = pdf_to_text(_path)

incorrect startxref pointer(3)


In [ ]:
page_nums_samenvatting = [i for i,p in enumerate(Text) if 'samenvatting' in p.lower()]
page_nums_dankwoord = [i for i,p in enumerate(Text) if any([t in p.lower() for t in ['dankwoord', 'nawoord']])]
page_nums_empty = [i for i,p in enumerate(Text) if p.strip()=='']
page_nums_ToC = [i for i,p in enumerate(Text) if 'content' in p.lower()]
print(page_nums_samenvatting, page_nums_dankwoord, page_nums_empty, page_nums_ToC, _File)
# extract the text from the pdfs based on the page_nums:
# if there is a page number, we assume that text on that page is relevant

# then we want to find the delimiters for the different sections

def get_samenvatting_page(txts, page_nums_ToC):
    ToC_num = min([n for n in page_nums_ToC if n>1])
    ToC_page = txts[ToC_num]
    return int(re.findall(r'Samenvatting[\s\t]+(\d+)', ToC_page))

# first element in ToC, after the samenvatting

